In [ ]:
import os 
from dotenv import load_dotenv

import os
import pandas as pd
import glob
import  numpy as np



In [ ]:
load_dotenv()

data_path=os.getenv("data_path")

## Steps in the code

1. Decision Making Function (decision_maker):

Once the 100 row (100 triggers tested for scores) the selected trigger is decided in the function. Selects DataFrame rows based on multiple criteria involving hanssen_kuipers_scores, bias_scores, and auroc_scores, returning rows with the maximum trigger value. This function is DEVELOPMENT. The current iteration of decision is as follows

   Steps:
1. Filters rows with the maximum hanssen_kuipers_scores and auroc_scores above 0.5.
2. Further filters by trigger_values greater than 0.1 and bias_scores below 1.0, selecting those with the maximum bias_score and auroc_scores above 0.5.
3. Additionally, selects rows with the maximum heidke_skill_scores and auroc_scores above 0.5.
4. Finally, from the combined filtered set, selects rows with the maximum trigger_values.


2. Environment Setup:

Imports necessary libraries (os, pandas, glob, numpy).
Loads environment variables using dotenv, specifically fetching the data_path.

3. Mapping Function (map_cell):

Splits a given cell value by _ and maps the first part to a corresponding value using a predefined dictionary.

4. Replacement Function (replace_with_list):

Replaces NaN float values with a predefined list of replacement values, otherwise returns the value unchanged.


5. Rounding Function (round_list):

Rounds each element in a list to a specified number of decimal places.

6. Data Processing:

Iterates through CSV files in the specified data path, applying decision_maker to filter and modify each DataFrame.
Merges the modified DataFrames, applies the map_cell function, and merges additional information from another CSV.

7. Pivoting and Final Adjustments:

Performs pivot operations to restructure the data for easier analysis and visualization.
Applies the replacement function to DataFrame cells and rounds off values as needed.

8. Exporting Results:

Saves the final processed and pivoted DataFrame to CSV files, one for POD (Probability of Detection) values and another for FAR (False Alarm Ratio).

10. Custom Mapping and Merging:

Utilizes a mapping_dict to associate cell values to regions and merges additional column information based on a unique identifier.

11. Final Data Preparation and Saving:

Prepares the data with specific columns, applies custom functions for value adjustments, and exports the final pivoted DataFrame to CSV files, indicating performance metrics for different regions and seasons. The resultant pod.csv and far.csv files are read and made into table plots in 06-plot-table. 


In [ ]:
def map_cell(cell_value):
    """
    Maps a cell value to a corresponding value using a predefined dictionary based on the first part of the cell value.

    Parameters:
    - cell_value (str): The value of the cell to be mapped, expected to contain parts separated by '_'.

    Returns:
    - The mapped value if the first part of `cell_value` is an integer key present in `mapping_dict`. Otherwise, returns None.

    Note:
    - This function relies on a global variable `mapping_dict` which should be a dictionary where keys are integers.
    """
    cell_parts = cell_value.split('_')
    if len(cell_parts) > 1:
        key = int(cell_parts[0])
        if key in mapping_dict:
            return mapping_dict[key]
    return None


def replace_with_list(x):
    """
    Replaces NaN float values with a predefined list of replacement values.

    Parameters:
    - x (float): The input value to be checked and potentially replaced.

    Returns:
    - A list of replacement values if `x` is a float and is NaN. Otherwise, returns `x` unchanged.

    Note:
    - This function is designed to handle cases where cell values in a dataset need to be replaced with a list of values
      for indicating missing or special cases.
    """
    replacement_values = [-999.0, -999.0]
    # If x is a float and it is nan (meaning the cell was originally empty), return the replacement list
    if isinstance(x, float) and np.isnan(x):
        return replacement_values
    # Otherwise, return x as it is
    return x


def decision_maker(df):
     """
    Selects rows from a DataFrame based on multiple criteria related to hanssen_kuipers_scores, bias_scores, and auroc_scores.

    The function performs several filtering steps to select rows that represent optimal conditions based on the specified metrics.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing score metrics.

    Returns:
    - tv_max (pandas.DataFrame): A DataFrame containing the rows that match the final criteria, particularly those with the maximum trigger value.

    Steps:
    1. Filters rows with the maximum hanssen_kuipers_scores and auroc_scores above 0.5.
    2. Further filters by trigger_values greater than 0.1 and bias_scores below 1.0, selecting those with the maximum bias_score and auroc_scores above 0.5.
    3. Additionally, selects rows with the maximum heidke_skill_scores and auroc_scores above 0.5.
    4. Finally, from the combined filtered set, selects rows with the maximum trigger_values.
    """
    hanssen_max_rows = df[(df['hanssen_kuipers_scores'] == df['hanssen_kuipers_scores'].max()) & (df['auroc_scores'] > 0.5)]
    df_filtered = df[df['trigger_values'] > 0.1]
    # Step 2: From those, further filter where bias score is below 1.0
    df_filtered = df_filtered[df_filtered['bias_scores'] < 1.0]
    # # Step 3: Now, select rows with the maximum bias score that also have auroc_scores above 0.5
    max_bias_below_one = df_filtered['bias_scores'].max()
    final_filtered_df = df_filtered[(df_filtered['bias_scores'] == max_bias_below_one) & (df_filtered['auroc_scores'] > 0.5)]
    heidke_max_rows = df[(df['heidke_skill_scores'] == df['heidke_skill_scores'].max()) & (df['auroc_scores'] > 0.5)]
    edf=pd.concat([hanssen_max_rows, final_filtered_df, heidke_max_rows])
    tv_max=edf[(edf['trigger_values'] == edf['trigger_values'].max())]
    #tv_max.to_csv(f'{output_path}{region_id}_{season_str}_{level}_lt{lead_int}.csv')
    return tv_max


def round_list(lst, decimal_places):
    """
    Rounds each element in a list to a specified number of decimal places.

    Parameters:
    - lst (list of float): The list of numbers to be rounded.
    - decimal_places (int): The number of decimal places to round each number to.

    Returns:
    - A list containing the rounded values of the input list.

    Note:
    - This function is useful for rounding numerical values in a list to ensure consistency or to improve readability.
    """
    return [round(x, decimal_places) for x in lst]


In [ ]:
mapping_dict = {0: 'Karamoja', 1: 'Marsabit', 2: 'Wajir'}

# List of CSV files
csv_files = glob.glob(f'{data_path}*.csv')

# Initialize an empty list to store DataFrames
dfs = []

# Read each CSV file into a DataFrame, add a new column with the file name, and append to dfs list
for file in csv_files:
    df = pd.read_csv(file)
    ddf=decision_maker(df)
    # Extract the file name without extension
    filename = os.path.splitext(os.path.basename(file))[0]
    # Add a new column with the file name
    ddf['filename'] = filename
    # Append the modified DataFrame to dfs list
    dfs.append(ddf)

# Concatenate all DataFrames row-wise
db = pd.concat(dfs, ignore_index=True)
db1=db.drop_duplicates('filename')

# Apply the mapping function to create a new column
db1['region_x'] = db1['filename'].apply(map_cell)


df1=pd.read_csv(f'{data_path}code_names.csv')


# Create a unique identifier in df1
df1['identifier'] = df1['region_id'].astype(str) + '_'+df1['season'] + '_' + df1['level'] + '_' + df1['lt']

db1['identifier'] = db1['filename']
db2 = db1.merge(df1[['identifier', 'col_name']], on='identifier', how='left')
db2['spi_prod_x'] = db2['filename'].str.split('_').str[1]
db3=db2[['trigger_values','hit_rates','false_alarm_ratios','region_x','col_name','spi_prod_x']]
db3['trigger_values']=db3['trigger_values']*100
db3['values'] = db3.apply(lambda x: [x['hit_rates'], x['trigger_values']], axis=1)



db3['values'] = db3['values'].apply(lambda x: round_list(x, 2))


pivoted_df = db3.pivot_table(index=['region_x', 'spi_prod_x'], 
                            columns='col_name', 
                            values='values', 
                            aggfunc='first')  # Assuming each combination is unique

# Reset the index to turn it back into columns
pivoted_df = pivoted_df.reset_index()


# Apply the custom function to each cell in the DataFrame
pivoted_df = pivoted_df.applymap(replace_with_list)

pivoted_df.columns.name = None

pivoted_df['empty1']=[[-999.0, -999.0]]* len(pivoted_df)
pivoted_df['empty2']=[[-999.0, -999.0]]* len(pivoted_df)


#efg_col=['region_x', 'spi_prod_x', 'nov_x', 'dec_x', 'jan_x', 'feb_x', 'mar_x', 'jun_x', 'jul_x', 'aug_x', 'sep_x', 'oct_x', 'nov_y', 'dec_y', 'jan_y', 'feb_y', 'mar_y', 'jun_y', 'jul_y', 'aug_y', 'sep_y', 'oct_y', 'nov', 'dec', 'jan', 'feb', 'mar', 'jun', 'jul', 'aug', 'sep', 'oct']

efg_col1=['region_x', 'spi_prod_x', 'nov_x', 'dec_x', 'jan_x',
       'feb_x', 'mar_x', 'apr_x', 'may_x', 'jun_x', 'jul_x', 'aug_x', 'sep_x',
       'oct_x', 'empty1', 'nov_y', 'dec_y', 'jan_y', 'feb_y', 'mar_y', 'apr_y',
       'may_y', 'jun_y', 'jul_y', 'aug_y', 'sep_y', 'oct_y', 'empty2', 'nov',
       'dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep',
       'oct']
pivoted_df1=pivoted_df[efg_col1]


pivoted_df1.to_csv(f'{data_path}pod.csv')




In [ ]:
db3['values'] = db3.apply(lambda x: [x['false_alarm_ratios'], x['trigger_values']], axis=1)

def round_list(lst, decimal_places):
    return [round(x, decimal_places) for x in lst]

db3['values'] = db3['values'].apply(lambda x: round_list(x, 2))


pivoted_df = db3.pivot_table(index=['region_x', 'spi_prod_x'], 
                            columns='col_name', 
                            values='values', 
                            aggfunc='first')  # Assuming each combination is unique

# Reset the index to turn it back into columns
pivoted_df = pivoted_df.reset_index()


# Apply the custom function to each cell in the DataFrame
pivoted_df = pivoted_df.applymap(replace_with_list)

pivoted_df.columns.name = None


pivoted_df['empty1']=[[-999.0, -999.0]]* len(pivoted_df)
pivoted_df['empty2']=[[-999.0, -999.0]]* len(pivoted_df)


#efg_col=['region_x', 'spi_prod_x', 'nov_x', 'dec_x', 'jan_x', 'feb_x', 'mar_x', 'jun_x', 'jul_x', 'aug_x', 'sep_x', 'oct_x', 'nov_y', 'dec_y', 'jan_y', 'feb_y', 'mar_y', 'jun_y', 'jul_y', 'aug_y', 'sep_y', 'oct_y', 'nov', 'dec', 'jan', 'feb', 'mar', 'jun', 'jul', 'aug', 'sep', 'oct']

efg_col1=['region_x', 'spi_prod_x', 'nov_x', 'dec_x', 'jan_x',
       'feb_x', 'mar_x', 'apr_x', 'may_x', 'jun_x', 'jul_x', 'aug_x', 'sep_x',
       'oct_x', 'empty1', 'nov_y', 'dec_y', 'jan_y', 'feb_y', 'mar_y', 'apr_y',
       'may_y', 'jun_y', 'jul_y', 'aug_y', 'sep_y', 'oct_y', 'empty2', 'nov',
       'dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep',
       'oct']
pivoted_df1=pivoted_df[efg_col1]


pivoted_df1.to_csv(f'{data_path}far.csv')